<a href="https://colab.research.google.com/github/Vitalyastiy/SQL-engineer/blob/main/Recursive_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Recursive

Мой канал тг: https://t.me/mikhaylovgleb

Это апгрейд sqlite3 чтобы были оконные функции. https://stackoverflow.com/a/59429952/4527289

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.50M/6.50M [00:00<00:00, 63.8MB/s]
mv: cannot move '_sqlite3.cpython-37m-x86_64-linux-gnu.so' to '/usr/lib/python3.7/lib-dynload/': No such file or directory


In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlite3

In [3]:
sqlite3.sqlite_version

'3.31.1'

In [4]:
con = sqlite3.connect('db')

In [5]:
cur = con.cursor()

In [6]:
def select(sql):
  return pd.read_sql(sql,con)

# Increment

In [7]:
sql = '''with rec(x) as (
select 1
union all
select t.x + 1 from rec t
where t.x < 10
)
select * from rec t'''

In [8]:
select(sql)

,x
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [9]:
select(sql)

,x
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


# Dates

In [10]:
sql = '''with dates(dt) as

(select date('2021-03-01')
union all
select date(t.dt,'+1 month') as dt from dates t
limit 10

)
select * from dates t'''

In [11]:
select(sql)

,dt
0,2021-03-01
1,2021-04-01
2,2021-05-01
3,2021-06-01
4,2021-07-01
5,2021-08-01
6,2021-09-01
7,2021-10-01
8,2021-11-01
9,2021-12-01


# Fibbonachi

1,1,2,3,5,8,13,21,34,...

In [12]:
sql = '''with fib(n1,n2,n3) as
(select null,1,1
union all
select 
t.n2 as n1, 
t.n3 as n2,
t.n2 + t.n3 as n3
from fib t
limit 10
)
select * from fib t'''

In [ ]:
select(sql)

,n1,n2,n3
0,NaN,1,1
1,1.0,1,2
2,1.0,2,3
3,2.0,3,5
4,3.0,5,8
5,5.0,8,13
6,8.0,13,21
7,13.0,21,34
8,21.0,34,55
9,34.0,55,89


# Running total

In [ ]:
df = pd.DataFrame({'id':[1,2,3,4,5,6],'num':[1,5,4,9,14,27]})

In [ ]:
df.to_sql('seq',con,index=False,if_exists='replace')

In [ ]:
df

,id,num
0,1,1
1,2,5
2,3,4
3,4,9
4,5,14
5,6,27


In [ ]:
sql = '''with running(id,num,rt) as 
(select t.id,t.num, t.num  from seq t
where t.id = 1
union all
select s.id,s.num, t.rt + s.num as rt from running t
join seq s on t.id = s.id - 1
limit 10
)
select * from running t'''

In [ ]:
select(sql)

,id,num,rt
0,1,1,1
1,2,5,6
2,3,4,10
3,4,9,19
4,5,14,33
5,6,27,60


# Accural

In [ ]:
sql = '''with running(id,num,rt) as 
(select t.id,t.num, t.num  from seq t
where t.id = 1
union all
select s.id,s.num,

case when t.rt > s.num then t.rt else
t.rt + s.num end as rt 

from running t
join seq s on t.id = s.id - 1
limit 10
)
select * from running t'''

In [ ]:
select(sql)

,id,num,rt
0,1,1,1
1,2,5,6
2,3,4,6
3,4,9,15
4,5,14,15
5,6,27,42


# Report to manager

https://leetcode.com/problems/all-people-report-to-the-given-manager

In [ ]:
sql = '''
drop table if exists employees;
Create table If Not Exists employees (employee_id int, employee_name varchar(30), manager_id int)
insert into Employees (employee_id, employee_name, manager_id) values ('1', 'Boss', '1')
insert into Employees (employee_id, employee_name, manager_id) values ('3', 'Alice', '3')
insert into Employees (employee_id, employee_name, manager_id) values ('2', 'Bob', '1')
insert into Employees (employee_id, employee_name, manager_id) values ('4', 'Daniel', '2')
insert into Employees (employee_id, employee_name, manager_id) values ('7', 'Luis', '4')
insert into Employees (employee_id, employee_name, manager_id) values ('8', 'John', '3')
insert into Employees (employee_id, employee_name, manager_id) values ('9', 'Angela', '8')
insert into Employees (employee_id, employee_name, manager_id) values ('77', 'Robert', '1')'''

In [ ]:
sql = ';\n'.join(sql.split('\n'))

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select * from employees t'''

In [ ]:
select(sql)

,employee_id,employee_name,manager_id
0,1,Boss,1
1,3,Alice,3
2,2,Bob,1
3,4,Daniel,2
4,7,Luis,4
5,8,John,3
6,9,Angela,8
7,77,Robert,1


Write an SQL query to find employee_id of all employees that directly or indirectly report their work to the head of the company.

The indirect relation between managers will not exceed 3 managers as the company is small.

In [ ]:
sql = '''with rec as 
(select t.*, 0 as depth from employees t
where t.employee_id = 1
union all
select e.employee_id, e.employee_name, e.manager_id, t.depth + 1 from rec t
join employees e on t.employee_id = e.manager_id and e.employee_id != 1
limit 100

)
select * from rec t
where t.depth <= 3'''

In [ ]:
select(sql)

,employee_id,employee_name,manager_id,depth
0,1,Boss,1,0
1,2,Bob,1,1
2,77,Robert,1,1
3,4,Daniel,2,2
4,7,Luis,4,3


# Gas Statition King

Само задание: https://colab.research.google.com/drive/1Y_btTY4qs3NP_F2qdarx_j5hCBKqsp8O?usp=sharing

In [ ]:
df = pd.DataFrame({'month': [1,2,3,4,5,6,7,8,9,10,11,12],
                   'gas': [30,50,50,50,50,50,50,30,50,50,50,50]})

In [ ]:
df.to_sql('gas',con,index=False,if_exists='replace')

In [ ]:
sql = '''
with flag as (
select t.*,

case when sum(case when t.gas >= 50 then 1 else 0 end)
over (order by t.month
ROWS BETWEEN 2 PRECEDING and CURRENT ROW
) = 3 then 1 else 0 end 
 as flag
 from gas t),
 bonus as (
 select t.*, 0 as n1, 0 as n2, 0 as n3 from flag t
 where t.month = 1

 union all
 select f.month, f.gas, f.flag,
 
 t.n2 as n1, t.n3 as n2, 

 case when t.n2 = 0 and t.n3 = 0 and f.flag = 1 then 1 else 0 end as n3

  from bonus t
 join flag f on t.month = f.month - 1

 limit 20
 
 )
 select t.month, t.gas, t.flag, t.n2 as prev_b,
 t.n1 as pre_prev_b, t.n3 as bonus from bonus t
 '''

In [ ]:
select(sql)

,month,gas,flag,prev_b,pre_prev_b,bonus
0,1,30,0,0,0,0
1,2,50,0,0,0,0
2,3,50,0,0,0,0
3,4,50,1,0,0,1
4,5,50,1,1,0,0
5,6,50,1,0,1,0
6,7,50,1,0,0,1
7,8,30,0,1,0,0
8,9,50,0,0,1,0
9,10,50,0,0,0,0
